<a href="https://colab.research.google.com/github/anandaru/GEN-AI/blob/main/Llama2_13B_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama 2
Hello people! Llama 2 is this awesome collection of pre-trained and fine-tuned generative text models, ranging from 7 billion to 70 billion parameters, all geared towards dialogue use cases.

What's really neat about it is that it beats most open-source chat models on benchmarks. Plus, when it comes to human evaluations for helpfulness and safety, it's right up there with the popular closed-source models.

One of the cool things within this collection is the [Llama 2 13B-chat](https://huggingface.co/meta-llama/Llama-2-13b-chat) model. And guess what? There's this neat tool called ***llama.cpp*** that's all about running the LLaMA model with 4-bit integer quantization right on your MacBook. It's built in plain C/C++, optimized for Apple silicon and x86 architectures, and supports various integer quantization and BLAS libraries.

Originally starting out as a web chat example, llama.cpp has evolved into this fantastic development playground for the ***ggml*** library features. And speaking of GGML, it's a C library for machine learning that makes distributing large language models (LLMs) a breeze.

GGML does some magic with quantization, which essentially means it enables efficient LLM execution on consumer hardware. The GGML files contain binary-encoded data, like version numbers, hyperparameters, vocabulary, and weights. The vocabulary is all about tokens for language generation, while the weights determine the LLM's size. And, of course, quantization reduces precision to optimize resource usage.

Exciting stuff, right? So whether you're diving into dialogue models or exploring the world of machine learning libraries, the Llama 2 and GGML are definitely worth checking out!

# Install all necessary packages.

In [ ]:

# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.4 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 14.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 13.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 39.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 35.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 8.9 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-g38v5n88/overlay/local/bin
    changing mode of /tmp/pip-build-env-g38v5n88/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-g38v5n88/overlay/

# Import all required libraries.

In [ ]:
from huggingface_hub import hf_hub_download

In [ ]:
from llama_cpp import Llama

# Download the model.

In [ ]:

model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

# Load the model.

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [ ]:

# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers


32

# Create a prompt template.

In [ ]:
prompt = "Write 10 test cases for the following requirement : An online HRMS portal on which the user logs in with their user account and password. The login page has two text fields for username and password. It also has two buttons – Login and Cancel.When successful, the login page directs the user to the HRMS home page. The cancel button cancels the login Specifications:The user id field requires a minimum of 6 characters, a maximum of 10 characters, numbers(0-9), letters(a-z, A-z), special characters (only underscore, period, hyphen allowed). It cannot be left blank. User id must begin with a number/character. It cannot include special characters.The password field requires a minimum of 6 characters, a maximum of 8 characters, numbers (0-9), letters (a-z, A-Z), all special characters. It cannot be blank."
prompt_template=f'''

PROMPT: {prompt}

ANSWER:
'''

# Generate the response.

In [ ]:
response=lcpp_llm(prompt=prompt_template, max_tokens=10000, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=3000,
                  echo=True)

Llama.generate: prefix-match hit


In [ ]:
print(response["choices"][0]["text"])



PROMPT: Write 10 test cases for the following requirement : An online HRMS portal on which the user logs in with their user account and password. The login page has two text fields for username and password. It also has two buttons – Login and Cancel.When successful, the login page directs the user to the HRMS home page. The cancel button cancels the login Specifications:The user id field requires a minimum of 6 characters, a maximum of 10 characters, numbers(0-9), letters(a-z, A-z), special characters (only underscore, period, hyphen allowed). It cannot be left blank. User id must begin with a number/character. It cannot include special characters.The password field requires a minimum of 6 characters, a maximum of 8 characters, numbers (0-9), letters (a-z, A-Z), all special characters. It cannot be blank.

ANSWER:
Test Case #1: Successful Login with Valid Username and Password
User ID: testuser123
Password: TestPass123
Expected Result: The user should be able to log in successfully 